# Closest Product Python Twitter Challenge

https://twitter.com/loicaroyer/status/1146173537658404864

Given a list of integers u=[a_0, ..., a_m]
find the set of indices {i_0, ..., i_n} for that list such that the product 
u[i_0]* ... *u[i_n] is the closest to a given integer N.
The shortest and most #elegant solution wins. 
(standard libs allowed)

## Setup

In [1]:
import numpy as np
import itertools as it

In [2]:
# Setup inputs
# Use same inputs as randomly generated by @james_t_webber
# https://twitter.com/james_t_webber/status/1146203142079410178
m = 20
N = 797
u = np.array([5,9,19,22,25,27,28,31,33,34,36,43,44,68,72,81,86,92,96,98])
print('N: ',N)
print('u: ',u)

N:  797
u:  [ 5  9 19 22 25 27 28 31 33 34 36 43 44 68 72 81 86 92 96 98]


## Brute Force Method
Calculate all the products and select the closest one. This takes on the order of seconds.

In [3]:
# Brute Force code by @james_t_webber adapted into a function
# This version computes the products of all combinations of u
# without repetition. Changed m = u.size
# https://twitter.com/james_t_webber/status/1146210405368266752
def closest_product_brute_force(N,u):
    i = min(
        map(np.array, it.product([False, True],repeat=u.size)),
        key=lambda i:np.abs(N-u[i].prod())
    )
    i = np.nonzero(i)[0]
    return i

In [4]:
%timeit closest_product_brute_force(N,u)
i = closest_product_brute_force(N,u)
print('i: ',i)
print('u[i]: ',u[i])
print('u[i].prod(): ',u[i].prod())

1 loop, best of 3: 9.46 s per loop
i:  [ 3 10]
u[i]:  [22 36]
u[i].prod():  792


## Speed-up Using Short Circuiting and Divisors
Rather than computing all the product of all the combinations, we just need to compute the products of the integers closest to N. Once we find such a product we can stop.

To do this, we iterate over the integers closest to N starting from N working outwards.
* Call each integer M.
* Figure out which elements of u are divisors of M.
* Compute the possible combinations of those divisors.
* Compute the product of those divisor combinations.
* Stop (short-circuit) if the product equals M
* Choose M one step farther from N

To implement this we will use a special type of iterable called a generator. Generators use lazy evaluation which defers evaluation until the next element of the iterable is needed. This will help us define the iteration without having to actually compute anything until we need it.

This takes 100s of microseconds, a 10^4 speed-up over the brute force method for the given input.

In [5]:
def closest_product_no_repetition(N,u):
    # Setup generator for product targets "M"
    Mg = it.chain.from_iterable(
        ( 
            (N-y,N+y) if y != 0 else (N,)
            for y in range(min(abs(N-u)))
        )
    )

    # For each M construct a list of divisors
    # Each generated element is a tuple: (divisors,M)
    divisors = filter(
        lambda f: f[0].size > 1,
        (
            (u[M % u == 0],M) for M in Mg
        )
    )


    # For each tuple of (divisors,M),
    # generate combinations of divisors
    divisor_combinations = it.chain.from_iterable(zip(
        it.chain.from_iterable(
            (it.combinations(d,r+1) for r in range(d.size))
        ),
        it.repeat(M)
    ) for d,M in divisors)

    # We want to use it.takewhile, but we need info on the divisor combination that met the condition
    # Use tee to duplicate the iterator and advance it by one
    g = it.tee(divisor_combinations)
    u_i = next(g[1])
    g = zip(*g)
    
    # Up to this point no products have been calculated, we have just set up iterators
    # Now, start iterating through M, divisors, and divisior_combinations
    # until the product equals M
    out = list(it.takewhile(lambda a: np.array(a[0][0]).prod() != a[0][1],g))
    if len(out) > 0:
        u_i = out[-1][1][0]
    else:
        u_i = list(u_i)[0]
        
    u_i = np.array(u_i)
    u_i = u_i[u_i != 1]
    i = list(map(lambda ui: np.where(u == ui)[0][0],u_i))
    return i

In [6]:
%timeit i = closest_product_no_repetition(N,u)
i = closest_product_no_repetition(N,u)
print('i: ',i)
print('u[i]: ',u[i])
print('u[i].prod(): ',u[i].prod())

1000 loops, best of 3: 195 µs per loop
i:  [3, 10]
u[i]:  [22 36]
u[i].prod():  792


## Repetition
It is not clear from the original problem description if elements of u can be repeated. This is easily addressed by just taking all the combinations with repetition.

In [7]:
def closest_product_with_repetition(N,u):
    # Setup generator for product targets "M"
    Mg = it.chain.from_iterable(
        ( 
            (N-y,N+y) if y != 0 else (N,)
            for y in range(min(abs(N-u)))
        )
    )

    # For each M construct a list of divisors
    # Each generated element is a tuple: (divisors,M)
    divisors = filter(
        lambda f: f[0].size > 1,
        (
            # Append 1 to the list of divisors so that we can use a fixed combination size
            (np.append(u[M % u == 0],1),M) for M in Mg
        )
    )

    # For each tuple of (divisors,M),
    # generate combinations of divisors with replacement
    divisor_combinations = it.chain.from_iterable(zip(
        it.combinations_with_replacement(
            d,
            # Since we allow for replacement,
            # set combination length such that a power of the smallest divisor exceeds M
            int(np.ceil(np.log(M)/np.log(min(d[d > 1]))))
        ),
        it.repeat(M)
    ) for d,M in divisors)

    # We want to use it.takewhile, but we need info on the divisor combination that met the condition
    # Use tee to duplicate the iterator and advance it by one
    g = it.tee(divisor_combinations)
    u_i = next(g[1])
    g = zip(*g)
    
    # Up to this point no products have been calculated, we have just set up iterators
    # Now, start iterating through M, divisors, and divisior_combinations
    # until the product equals M
    out = list(it.takewhile(lambda a: np.array(a[0][0]).prod() != a[0][1],g))
    if len(out) > 0:
        u_i = out[-1][1][0]
    else:
        u_i = list(u_i)[0]
        
    u_i = np.array(u_i)
    u_i = u_i[u_i != 1]
    i = list(map(lambda ui: np.where(u == ui)[0][0],u_i))
    return i

In [8]:
%timeit i = closest_product_with_repetition(N,u)
i = closest_product_with_repetition(N,u)
print('i: ',i)
print('u[i]: ',u[i])
print('u[i].prod(): ',u[i].prod())

1000 loops, best of 3: 1.26 ms per loop
i:  [3, 10]
u[i]:  [22 36]
u[i].prod():  792


## Breakdown of the method

First we define a helper function, peek_with_label to help take a look at some of the generators used.

In [9]:
def peek_with_label(iterable,num=10,label=''):
    iterable = it.tee(iterable)
    print(label)
    for x in it.islice(iterable[0],num):
        print(x)
    return iterable[1]

In [10]:
# Setup generator for product targets "M"
Mg = it.chain.from_iterable(
    ( 
        (N-y,N+y) if y != 0 else (N,)
        for y in range(min(abs(N-u)))
    )
)
Mg = peek_with_label(Mg,10,'M generator:')

M generator:
797
796
798
795
799
794
800
793
801
792


In [11]:
# For each M construct a list of divisors
# Each generated element is a tuple: (divisors,M)
divisors = filter(
    lambda f: f[0].size > 1,
    (
        # Append 1 to the list of divisors so that we can use a fixed combination size
        (np.append(u[M % u == 0],1),M) for M in Mg
    )
)

divisors = peek_with_label(divisors,10,'Divisors: ')

Divisors: 
(array([19,  1]), 798)
(array([5, 1]), 795)
(array([ 5, 25,  1]), 800)
(array([9, 1]), 801)
(array([ 9, 22, 33, 36, 44, 72,  1]), 792)
(array([5, 1]), 790)
(array([5, 1]), 805)
(array([31,  1]), 806)
(array([5, 1]), 785)
(array([28, 98,  1]), 784)


In [12]:
# For each tuple of (divisors,M),
# generate combinations of divisors with replacement
divisor_combinations = it.chain.from_iterable(zip(
    it.combinations_with_replacement(
        d,
        # Since we allow for replacement,
        # set combination length such that a power of the smallest divisor exceeds M
        int(np.ceil(np.log(M)/np.log(min(d[d > 1]))))
    ),
    it.repeat(M)
) for d,M in divisors)

divisor_combinations = peek_with_label(divisor_combinations,240,'Divisor combinations with repetition: ')

Divisor combinations with repetition: 
((19, 19, 19), 798)
((19, 19, 1), 798)
((19, 1, 1), 798)
((1, 1, 1), 798)
((5, 5, 5, 5, 5), 795)
((5, 5, 5, 5, 1), 795)
((5, 5, 5, 1, 1), 795)
((5, 5, 1, 1, 1), 795)
((5, 1, 1, 1, 1), 795)
((1, 1, 1, 1, 1), 795)
((5, 5, 5, 5, 5), 800)
((5, 5, 5, 5, 25), 800)
((5, 5, 5, 5, 1), 800)
((5, 5, 5, 25, 25), 800)
((5, 5, 5, 25, 1), 800)
((5, 5, 5, 1, 1), 800)
((5, 5, 25, 25, 25), 800)
((5, 5, 25, 25, 1), 800)
((5, 5, 25, 1, 1), 800)
((5, 5, 1, 1, 1), 800)
((5, 25, 25, 25, 25), 800)
((5, 25, 25, 25, 1), 800)
((5, 25, 25, 1, 1), 800)
((5, 25, 1, 1, 1), 800)
((5, 1, 1, 1, 1), 800)
((25, 25, 25, 25, 25), 800)
((25, 25, 25, 25, 1), 800)
((25, 25, 25, 1, 1), 800)
((25, 25, 1, 1, 1), 800)
((25, 1, 1, 1, 1), 800)
((1, 1, 1, 1, 1), 800)
((9, 9, 9, 9), 801)
((9, 9, 9, 1), 801)
((9, 9, 1, 1), 801)
((9, 1, 1, 1), 801)
((1, 1, 1, 1), 801)
((9, 9, 9, 9), 792)
((9, 9, 9, 22), 792)
((9, 9, 9, 33), 792)
((9, 9, 9, 36), 792)
((9, 9, 9, 44), 792)
((9, 9, 9, 72), 792)
((9, 9

In [13]:
# We want to use it.takewhile, but we need info on the divisor combination that met the condition
# Use tee to duplicate the iterator and advance it by one
g = it.tee(divisor_combinations)
u_i = next(g[1])
g = zip(*g)

g = peek_with_label(g,10,'Paired divisor combinations:')

Paired divisor combinations:
(((19, 19, 19), 798), ((19, 19, 1), 798))
(((19, 19, 1), 798), ((19, 1, 1), 798))
(((19, 1, 1), 798), ((1, 1, 1), 798))
(((1, 1, 1), 798), ((5, 5, 5, 5, 5), 795))
(((5, 5, 5, 5, 5), 795), ((5, 5, 5, 5, 1), 795))
(((5, 5, 5, 5, 1), 795), ((5, 5, 5, 1, 1), 795))
(((5, 5, 5, 1, 1), 795), ((5, 5, 1, 1, 1), 795))
(((5, 5, 1, 1, 1), 795), ((5, 1, 1, 1, 1), 795))
(((5, 1, 1, 1, 1), 795), ((1, 1, 1, 1, 1), 795))
(((1, 1, 1, 1, 1), 795), ((5, 5, 5, 5, 5), 800))


In [14]:
# Up to this point no products have been calculated, we have just set up iterators
# Now, start iterating through M, divisors, and divisior_combinations
# until the product equals M
out = list(it.takewhile(lambda a: np.array(a[0][0]).prod() != a[0][1],g))

for x in out[-10:]:
    print(x)

(((22, 33, 1, 1), 792), ((22, 36, 36, 36), 792))
(((22, 36, 36, 36), 792), ((22, 36, 36, 44), 792))
(((22, 36, 36, 44), 792), ((22, 36, 36, 72), 792))
(((22, 36, 36, 72), 792), ((22, 36, 36, 1), 792))
(((22, 36, 36, 1), 792), ((22, 36, 44, 44), 792))
(((22, 36, 44, 44), 792), ((22, 36, 44, 72), 792))
(((22, 36, 44, 72), 792), ((22, 36, 44, 1), 792))
(((22, 36, 44, 1), 792), ((22, 36, 72, 72), 792))
(((22, 36, 72, 72), 792), ((22, 36, 72, 1), 792))
(((22, 36, 72, 1), 792), ((22, 36, 1, 1), 792))


In [15]:
    if len(out) > 0:
        u_i = out[-1][1][0]
    else:
        u_i = list(u_i)[0]
        
    u_i = np.array(u_i)
    u_i = u_i[u_i != 1]
    i = list(map(lambda ui: np.where(u == ui)[0][0],u_i))

In [16]:
print('i: ',i)
print('u[i]: ',u[i])
print('u[i].prod(): ',u[i].prod())

i:  [3, 10]
u[i]:  [22 36]
u[i].prod():  792


## Unified Function with or without repetition

In [17]:
def closest_product(N,u,repetition=False):
    # Process input
    u = np.array(u)
    # Setup generator for product targets "M"
    Mg = it.chain.from_iterable(
        ( 
            (N-y,N+y) if y != 0 else (N,)
            for y in range(min(abs(N-u)))
        )
    )

    if repetition:
        # For each M construct a list of divisors
        # Each generated element is a tuple: (divisors,M)
        divisors = filter(
            lambda f: f[0].size > 1,
            (
                # Append 1 to the list of divisors so that we can use a fixed combination size
                (np.append(u[M % u == 0],1),M) for M in Mg
            )
        )
        # For each tuple of (divisors,M),
        # generate combinations of divisors with replacement
        divisor_combinations = it.chain.from_iterable(zip(
            it.combinations_with_replacement(
                d,
                # Since we allow for replacement,
                # set combination length such that a power of the smallest divisor exceeds M
                int(np.ceil(np.log(M)/np.log(min(d[d > 1]))))
            ),
            it.repeat(M)
        ) for d,M in divisors)
    else:
        # For each M construct a list of divisors
        # Each generated element is a tuple: (divisors,M)
        divisors = filter(
            lambda f: f[0].size > 1,
            (
                (u[M % u == 0],M) for M in Mg
            )
        )
        # For each tuple of (divisors,M),
        # generate combinations of divisors
        divisor_combinations = it.chain.from_iterable(zip(
            it.chain.from_iterable(
                (it.combinations(d,r+1) for r in range(d.size))
            ),
            it.repeat(M)
        ) for d,M in divisors)

    # We want to use it.takewhile, but we need info on the divisor combination that met the condition
    # Use tee to duplicate the iterator and advance it by one
    g = it.tee(divisor_combinations)
    u_i = next(g[1])
    g = zip(*g)
    
    # Up to this point no products have been calculated, we have just set up iterators
    # Now, start iterating through M, divisors, and divisior_combinations
    # until the product equals M
    out = list(it.takewhile(lambda a: np.array(a[0][0]).prod() != a[0][1],g))
    if len(out) > 0:
        u_i = out[-1][1][0]
    else:
        u_i = list(u_i)[0]
        
    u_i = np.array(u_i)
    u_i = u_i[u_i != 1]
    i = list(map(lambda ui: np.where(u == ui)[0][0],u_i))
    return i

### Unified Function testing

In [18]:
closest_product(8,[2,3],repetition=False)

[0, 1]

In [19]:
closest_product(8,np.array([2,3]),repetition=True)

[0, 0, 0]

In [20]:
closest_product(N,u,repetition=False)

[3, 10]

In [21]:
closest_product(N,u,repetition=True)

[3, 10]

## Create a random challenging case, with m = 100

If we create a random challenging case, executing time is in the milliseconds

In [22]:
challenging_N = np.random.randint(101,2**20)
challenging_u = np.random.choice(range(2,1010),100,False)
print('challenging_N: ',challenging_N)
print('challenging_u: ',challenging_u)

challenging_N:  621229
challenging_u:  [ 338  722  335  100  955  272  514  516  172  661  757  877  617  874
  237  328  953  901  552  694  844  933  574  508  116  428  336  752
  949  207  473  366  220  185  222  262   23  485  488  114   77  343
   37  843  490  622  466  259  110  554  520  837  352   48  501  735
  278  450  873  792   66  919  804  153  619  798  243  681  144 1001
  412  351  737  635  715  238  697  911  480  182  576   47  643  788
  394  651  571  492  905  218  414  678  383  314  660  393  588  459
  638  124]


In [23]:
%time i = closest_product(challenging_N,challenging_u,repetition=False)
i = closest_product(challenging_N,challenging_u,repetition=False)
print('i: ',i)
print('u[i]: ',challenging_u[i])
print('u[i].prod(): ',challenging_u[i].prod())

CPU times: user 2.99 ms, sys: 1 ms, total: 3.99 ms
Wall time: 4.01 ms
i:  [43, 72]
u[i]:  [843 737]
u[i].prod():  621291
